In [1]:
import csv
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
import numpy as np 


In [2]:
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
crime_data = pd.read_csv("Crimes_-_2001_to_Present.csv", encoding="ISO-8859-1")

In [3]:
# Preview the first 5 lines of the loaded data 

sample_crime_data = crime_data.sample(n = 700000) 
sample_crime_data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
6832089,9463992,HX116607,01/06/2014 01:00:00 PM,118XX S AVENUE O,0820,THEFT,$500 AND UNDER,SMALL RETAIL STORE,False,False,...,10.0,55.0,06,1200937.0,1827157.0,2014,02/04/2016 06:33:39 AM,41.680482,-87.540100,"(41.680481664, -87.540099953)"
6103270,8297082,HT530543,10/06/2011 04:00:00 PM,047XX S KILDARE AVE,0320,ROBBERY,STRONGARM - NO WEAPON,"SCHOOL, PUBLIC, GROUNDS",False,False,...,23.0,57.0,03,1148489.0,1872954.0,2011,02/10/2018 03:50:01 PM,41.807322,-87.730913,"(41.807321581, -87.730912667)"
642828,10506939,HZ248684,05/02/2016 05:00:00 PM,021XX W PETERSON AVE,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,True,False,...,40.0,2.0,06,1160994.0,1939841.0,2016,02/10/2018 03:50:01 PM,41.990615,-87.683189,"(41.990615196, -87.683188577)"
5380833,7168556,HR578203,10/08/2009 06:10:00 PM,015XX S BLUE ISLAND AVE,031B,ROBBERY,ARMED: OTHER FIREARM,SIDEWALK,False,False,...,25.0,28.0,03,1168118.0,1892716.0,2009,02/28/2018 03:56:25 PM,41.861150,-87.658349,"(41.861150316, -87.658349464)"
1261972,11428199,JB410567,08/27/2018 03:20:00 AM,088XX S HALSTED ST,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,False,False,...,21.0,71.0,05,1172525.0,1846212.0,2018,09/03/2018 04:08:41 PM,41.733442,-87.643542,"(41.733442201, -87.643542193)"


In [4]:
# Description of crime data
sample_crime_data.describe()

,ID,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Year,Latitude,Longitude
count,7.000000e+05,700000.000000,699993.000000,639662.000000,639797.000000,6.932070e+05,6.932070e+05,700000.000000,693207.000000,693207.000000
mean,6.535509e+06,1188.080460,11.285654,22.679987,37.560426,1.164565e+06,1.885727e+06,2008.924974,41.842031,-87.671628
std,3.219002e+06,703.191197,6.946648,13.844642,21.539272,1.696024e+04,3.241276e+04,5.463390,0.089181,0.061446
min,6.370000e+02,111.000000,1.000000,1.000000,0.000000,0.000000e+00,0.000000e+00,2001.000000,36.619446,-91.686566
25%,3.556328e+06,622.000000,6.000000,10.000000,23.000000,1.152956e+06,1.859152e+06,2004.000000,41.768858,-87.713751
50%,6.532558e+06,1034.000000,10.000000,22.000000,32.000000,1.166069e+06,1.890625e+06,2008.000000,41.855655,-87.665926
75%,9.321595e+06,1731.000000,17.000000,34.000000,57.000000,1.176363e+06,1.909175e+06,2013.000000,41.906593,-87.628275
max,1.209561e+07,2535.000000,31.000000,50.000000,77.000000,1.205119e+06,1.951532e+06,2020.000000,42.022644,-87.524529


In [5]:
uniq_crime_type = sample_crime_data['Primary Type'].unique() 
uniq_crime_type

# crime_data.columns

# Description

array(['THEFT', 'ROBBERY', 'BURGLARY', 'BATTERY', 'NARCOTICS',
       'CRIM SEXUAL ASSAULT', 'ASSAULT', 'CRIMINAL DAMAGE',
       'OTHER OFFENSE', 'DECEPTIVE PRACTICE', 'MOTOR VEHICLE THEFT',
       'CRIMINAL TRESPASS', 'WEAPONS VIOLATION',
       'OFFENSE INVOLVING CHILDREN', 'PUBLIC PEACE VIOLATION',
       'LIQUOR LAW VIOLATION', 'PROSTITUTION', 'INTIMIDATION',
       'OTHER NARCOTIC VIOLATION', 'INTERFERENCE WITH PUBLIC OFFICER',
       'SEX OFFENSE', 'HOMICIDE', 'GAMBLING', 'OBSCENITY',
       'CRIMINAL SEXUAL ASSAULT', 'CONCEALED CARRY LICENSE VIOLATION',
       'KIDNAPPING', 'ARSON', 'STALKING', 'PUBLIC INDECENCY',
       'HUMAN TRAFFICKING', 'NON-CRIMINAL (SUBJECT SPECIFIED)',
       'NON-CRIMINAL', 'RITUALISM', 'NON - CRIMINAL'], dtype=object)

In [6]:
uniq_crime_desc = crime_data['FBI Code'].unique() 
uniq_crime_desc

array(['11', '02', '05', '06', '17', '14', '26', '20', '08B', '12', '08A',
       '18', '10', '07', '03', '13', '15', '04B', '04A', '22', '16', '24',
       '09', '19', '01A', '01B'], dtype=object)

In [7]:
# crime data without null value in Longitude ... assume the same null for latitude
not_null_crime_data = sample_crime_data[sample_crime_data['Longitude'].notnull()]
not_null_crime_data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
6832089,9463992,HX116607,01/06/2014 01:00:00 PM,118XX S AVENUE O,0820,THEFT,$500 AND UNDER,SMALL RETAIL STORE,False,False,...,10.0,55.0,06,1200937.0,1827157.0,2014,02/04/2016 06:33:39 AM,41.680482,-87.540100,"(41.680481664, -87.540099953)"
6103270,8297082,HT530543,10/06/2011 04:00:00 PM,047XX S KILDARE AVE,0320,ROBBERY,STRONGARM - NO WEAPON,"SCHOOL, PUBLIC, GROUNDS",False,False,...,23.0,57.0,03,1148489.0,1872954.0,2011,02/10/2018 03:50:01 PM,41.807322,-87.730913,"(41.807321581, -87.730912667)"
642828,10506939,HZ248684,05/02/2016 05:00:00 PM,021XX W PETERSON AVE,0860,THEFT,RETAIL THEFT,DEPARTMENT STORE,True,False,...,40.0,2.0,06,1160994.0,1939841.0,2016,02/10/2018 03:50:01 PM,41.990615,-87.683189,"(41.990615196, -87.683188577)"
5380833,7168556,HR578203,10/08/2009 06:10:00 PM,015XX S BLUE ISLAND AVE,031B,ROBBERY,ARMED: OTHER FIREARM,SIDEWALK,False,False,...,25.0,28.0,03,1168118.0,1892716.0,2009,02/28/2018 03:56:25 PM,41.861150,-87.658349,"(41.861150316, -87.658349464)"
1261972,11428199,JB410567,08/27/2018 03:20:00 AM,088XX S HALSTED ST,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,False,False,...,21.0,71.0,05,1172525.0,1846212.0,2018,09/03/2018 04:08:41 PM,41.733442,-87.643542,"(41.733442201, -87.643542193)"


In [8]:
# Added column "Crime" with FBI Uniform crime report

not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '01A', 'Crime'] = ' Murder and Nonnegligent Manslaughter'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '01B', 'Crime'] = 'Manslaughter by Negligence'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '02', 'Crime'] = 'Forcible Rape'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '03', 'Crime'] = 'Robbery'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '04A', 'Crime'] = 'Aggravated Assaul- Firearm'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '04B', 'Crime'] = 'Aggravated Assaul-Knife or Cutting Instrument'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '05', 'Crime'] = 'Burglary'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '06', 'Crime'] = 'Larceny-thefT'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '07', 'Crime'] = 'Motor Vehicle Thef'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '08A', 'Crime'] = 'Arson-Structural'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '08B', 'Crime'] = 'Arson-Structural'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '09', 'Crime'] = 'Other Assaults'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '10', 'Crime'] = 'Forgery and Counterfeiting'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '12', 'Crime'] = 'Embezzlement'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '13', 'Crime'] = 'Stolen Property: Buying, Receiving, Possessing'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '14', 'Crime'] = 'Vandalism'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '15', 'Crime'] = 'Weapons: Carrying, Possessing, etc'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '16', 'Crime'] = ' Prostitution and Commercialized Vice'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '17', 'Crime'] = 'Sex Offenses'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '18', 'Crime'] = 'Drug Abuse Violations'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '19', 'Crime'] = 'Gambling'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '20', 'Crime'] = 'Offenses Against the Family and Children'
not_null_crime_data.loc[not_null_crime_data['FBI Code'] == '26', 'Crime'] = 'All Other Offenses'



not_null_crime_data.loc[not_null_crime_data['Primary Type'] == 'BATTERY', 'Primary Type'] = 'ASSAULT'

In [9]:
uniq_crime_type = not_null_crime_data['Primary Type'].unique() 
uniq_crime_type

array(['THEFT', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'NARCOTICS',
       'CRIM SEXUAL ASSAULT', 'CRIMINAL DAMAGE', 'OTHER OFFENSE',
       'DECEPTIVE PRACTICE', 'MOTOR VEHICLE THEFT', 'CRIMINAL TRESPASS',
       'WEAPONS VIOLATION', 'OFFENSE INVOLVING CHILDREN',
       'PUBLIC PEACE VIOLATION', 'LIQUOR LAW VIOLATION', 'PROSTITUTION',
       'INTIMIDATION', 'OTHER NARCOTIC VIOLATION',
       'INTERFERENCE WITH PUBLIC OFFICER', 'SEX OFFENSE', 'HOMICIDE',
       'GAMBLING', 'OBSCENITY', 'CRIMINAL SEXUAL ASSAULT',
       'CONCEALED CARRY LICENSE VIOLATION', 'KIDNAPPING', 'ARSON',
       'STALKING', 'PUBLIC INDECENCY', 'HUMAN TRAFFICKING',
       'NON-CRIMINAL (SUBJECT SPECIFIED)', 'NON-CRIMINAL', 'RITUALISM',
       'NON - CRIMINAL'], dtype=object)

In [10]:
# Describe filtered data 
not_null_crime_data.describe()

# not_null_crime_data.head()

,ID,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Year,Latitude,Longitude
count,6.932070e+05,693207.000000,693200.000000,633817.000000,633945.000000,6.932070e+05,6.932070e+05,693207.000000,693207.000000,693207.000000
mean,6.531973e+06,1187.878684,11.284657,22.672047,37.574297,1.164565e+06,1.885727e+06,2008.929940,41.842031,-87.671628
std,3.213609e+06,703.080553,6.945378,13.842510,21.538375,1.696024e+04,3.241276e+04,5.458525,0.089181,0.061446
min,6.370000e+02,111.000000,1.000000,1.000000,0.000000,0.000000e+00,0.000000e+00,2001.000000,36.619446,-91.686566
25%,3.561848e+06,622.000000,6.000000,10.000000,23.000000,1.152956e+06,1.859152e+06,2004.000000,41.768858,-87.713751
50%,6.526372e+06,1034.000000,10.000000,22.000000,32.000000,1.166069e+06,1.890625e+06,2008.000000,41.855655,-87.665926
75%,9.307439e+06,1731.000000,17.000000,34.000000,57.000000,1.176363e+06,1.909175e+06,2013.000000,41.906593,-87.628275
max,1.209373e+07,2535.000000,31.000000,50.000000,77.000000,1.205119e+06,1.951532e+06,2020.000000,42.022644,-87.524529


In [11]:
not_null_crime_data.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location', 'Crime'],
      dtype='object')

In [12]:
# new = old.filter(['A','B','D'], axis=1)

final_crime_data = not_null_crime_data.filter(['ID','Date','Primary Type', 'Latitude','Longitude' ], axis=1)
final_crime_data.describe()


,ID,Latitude,Longitude
count,6.932070e+05,693207.000000,693207.000000
mean,6.531973e+06,41.842031,-87.671628
std,3.213609e+06,0.089181,0.061446
min,6.370000e+02,36.619446,-91.686566
25%,3.561848e+06,41.768858,-87.713751
50%,6.526372e+06,41.855655,-87.665926
75%,9.307439e+06,41.906593,-87.628275
max,1.209373e+07,42.022644,-87.524529


In [13]:
final_crime_data.rename(columns={'Primary Type': 'Crime'}, inplace=True)
final_crime_data

,ID,Date,Crime,Latitude,Longitude
6832089,9463992,01/06/2014 01:00:00 PM,THEFT,41.680482,-87.540100
6103270,8297082,10/06/2011 04:00:00 PM,ROBBERY,41.807322,-87.730913
642828,10506939,05/02/2016 05:00:00 PM,THEFT,41.990615,-87.683189
5380833,7168556,10/08/2009 06:10:00 PM,ROBBERY,41.861150,-87.658349
1261972,11428199,08/27/2018 03:20:00 AM,BURGLARY,41.733442,-87.643542
...,...,...,...,...,...
2188687,2269185,08/01/2002 02:40:00 AM,BURGLARY,41.710201,-87.671156
2272335,2373180,09/28/2002 10:00:31 PM,ASSAULT,41.792445,-87.622407
1045095,11131500,10/26/2017 06:00:00 AM,OTHER OFFENSE,41.973458,-87.751628
338409,12089032,06/26/2020 04:30:00 PM,ASSAULT,41.722581,-87.628687


In [15]:
# Create the connection using the imported create_engine function and 
# then invoking the connect method on it.
engine = create_engine('sqlite:///Chicagocrime.db', echo=True)
sqlite_connection = engine.connect()


2020-07-10 03:07:53,190 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-10 03:07:53,193 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 03:07:53,194 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-10 03:07:53,195 INFO sqlalchemy.engine.base.Engine ()


In [17]:
sqlite_table = "criminals_mapps"
final_crime_data.to_sql(sqlite_table, sqlite_connection, if_exists='fail')
sqlite_connection.close()

2020-07-10 03:08:34,377 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("criminals_mapps")
2020-07-10 03:08:34,378 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 03:08:34,379 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("criminals_mapps")
2020-07-10 03:08:34,379 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 03:08:34,381 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE criminals_mapps (
	"index" BIGINT, 
	"ID" BIGINT, 
	"Date" TEXT, 
	"Crime" TEXT, 
	"Latitude" FLOAT, 
	"Longitude" FLOAT
)


2020-07-10 03:08:34,382 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 03:08:34,499 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-10 03:08:34,499 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_criminals_mapps_index ON criminals_mapps ("index")
2020-07-10 03:08:34,500 INFO sqlalchemy.engine.base.Engine ()
2020-07-10 03:08:34,590 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-10 03:08:34,852 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-10 03:08:3